In [1]:
import pandas as pd
import numpy as np

# read in all files from the folder called ../data
heart_table = pd.read_csv('../data/dev_sample_HeartTable_10_17_2022.csv')
medical_source = pd.read_csv('../data/CF_medicalData.csv')
agr_cell_email = pd.read_csv('../data/cf_agrEmailCellAppend_va2_10_14_2022.csv')
acxiom_email = pd.read_csv('../data/cf_acxiomEmails_va2_10_17_2022.csv')


C:\Users\cayde\AppData\Local\Temp\ipykernel_13968\1339307102.py:6: DtypeWarning: Columns (16,17,18,19,20,21,22,23,34,36,49,50,51,52,53,54,55,56,57,58,59,60,62,63,64,74,75,76,78,79,80,85,87,88,89,90,91,92,93,97,98,99,108,123,124) have mixed types. Specify dtype option on import or set low_memory=False.
  medical_source = pd.read_csv('../data/CF_medicalData.csv')
C:\Users\cayde\AppData\Local\Temp\ipykernel_13968\1339307102.py:7: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  agr_cell_email = pd.read_csv('../data/cf_agrEmailCellAppend_va2_10_14_2022.csv')


In [4]:
#count the number of rows in heart table that in the column clearforce id there is the substring "medical_source"
print(heart_table[heart_table['source_id'].str.contains('medical_source')].shape[0])

#count the number of rows in medial source that in the column clearforce id that are not null
print(medical_source[medical_source['source_id'].notnull()].shape[0])

11193
11193


In [2]:
#count the number of duplicates in the column 'source_id'
print(heart_table['source_id'].duplicated().sum())

0


# get all causes of death

In [2]:
# drop the columns that are not needed 'AGRID15'
heart_table = heart_table.drop(columns=['AGRID15', 'AGRID16'])

In [3]:
#make a subset of heart_table where the column source_id contains 'V1_population2_sample'
v1_pop = heart_table[heart_table['source_id'].str.contains('V1_population2_sample')]

#make a subset of medical_source
medical_source_sub = medical_source[['Immediate Cause A', 'ENDate', 'LOD Date', 'FirstName', 'LastName', 'BirthDate']]

# merge v1_pop with medical_source on the columns FIRST_NAME, LAST_NAME, and DOB
v1_pop_medical = pd.merge(v1_pop, medical_source_sub, left_on=['FIRST_NAME', 'LAST_NAME', 'MASTER_DOB'], right_on=['FirstName', 'LastName', 'BirthDate'], how='left')

v1_pop_medical.rename(columns={'source_id_x': 'source_id'}, inplace=True)

In [4]:
#cut down med df to 'source_id' and 'Immediate Cause A' 
v1_pop_medical = v1_pop_medical[['source_id', 'Immediate Cause A', 'ENDate', 'LOD Date']]
medical_source = medical_source[['source_id', 'Immediate Cause A', 'ENDate', 'LOD Date']]


In [5]:
#merge heart_table and medical_source on source_id
merge_master = pd.merge(heart_table, medical_source, on='source_id', how='left')
merge_master = pd.merge(merge_master, v1_pop_medical, on='source_id', how='left')


In [6]:
# if the column 'Immediate Cause A_x' is null, then replace it with the value in 'Immediate Cause A_y'
merge_master['Immediate Cause A_x'] = np.where(merge_master['Immediate Cause A_x'].isnull(), merge_master['Immediate Cause A_y'], merge_master['Immediate Cause A_x'])
#drop the column 'Immediate Cause A_y'
merge_master = merge_master.drop(columns=['Immediate Cause A_y'])
#rename the column 'Immediate Cause A_x' to 'Immediate Cause A'
merge_master.rename(columns={'Immediate Cause A_x': 'Immediate Cause A'}, inplace=True)

# if the column 'ENDate_x' is null, then replace it with the value in 'ENDate_y'
merge_master['ENDate_x'] = np.where(merge_master['ENDate_x'].isnull(), merge_master['ENDate_y'], merge_master['ENDate_x'])
#drop the column 'ENDate_y'
merge_master = merge_master.drop(columns=['ENDate_y'])
#rename the column 'ENDate_x' to 'ENDate'
merge_master.rename(columns={'ENDate_x': 'ENDate'}, inplace=True)

# if the column 'LOD Date_x' is null, then replace it with the value in 'LOD Date_y'
merge_master['LOD Date_x'] = np.where(merge_master['LOD Date_x'].isnull(), merge_master['LOD Date_y'], merge_master['LOD Date_x'])
#drop the column 'LOD Date_y'
merge_master = merge_master.drop(columns=['LOD Date_y'])
#rename the column 'LOD Date_x' to 'LOD Date'
merge_master.rename(columns={'LOD Date_x': 'LOD Date'}, inplace=True)

# get all emails and cells

In [7]:
#cut down agr_cell_email to 'source_id' and 'EMAIL1', 'EMAIL2', 'EMAIL3', Cell_Phone1', 'Cell_Phone2', 'Cell_Phone3'
agr_cell_email = agr_cell_email[['source_id', 'EMAIL1', 'EMAIL2', 'EMAIL3', 'Cell_Phone1', 'Cell_Phone2', 'Cell_Phone3']]

In [8]:
acxiom_email = acxiom_email[['source_id', 'person.emailAppend0.email']]

In [9]:
#merge heart_table and medical_source on source_id
merge_master = pd.merge(merge_master, agr_cell_email, on='source_id', how='left')

#count the number of duplicates in the column 'source_id'
print(merge_master['source_id'].duplicated().sum())
#store the duplicates in a variable
duplicates = merge_master[merge_master['source_id'].duplicated()]

#remove all but the first duplicate
merge_master = merge_master.drop_duplicates(subset='source_id', keep='first')

6


In [10]:
merge_master = pd.merge(merge_master, acxiom_email, on='source_id', how='left')
merge_master = merge_master.drop_duplicates(subset='source_id', keep='first')

In [11]:
#count the number of with a non null value in the column 'Immediate Cause A'
merge_master['Immediate Cause A'].count()

11737

In [12]:
#writing the file to a csv
merge_master.to_csv('../data/out/ThreatMinder_22ksample_cf_v2_10_18_2022.csv', index=False)


In [13]:
#write to a csv file the dataframe v1_pop_medical
v1_pop_medical.to_csv('../data/out/v1sample_causeofdeath_v2_10_18_2022.csv', index=False)